In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import site

import pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import rioxarray as rxr

import cryogrid_pytools as cg
from cryogrid_pytools import data as cg_data
import cryogrid_run_manager as cgrm

In [3]:
bbox_WSEN = 71.537796,39.627680,71.680705,39.707624

In [6]:
cgrm.templater.main(
    '../runs/abramov-tmp/',
    bbox_WSEN=bbox_WSEN,
    template_dir='../templates/',
    n_land_classes=3,
)

2025-03-21 10:19:06.774 | INFO     | cryogrid_pytools.excel_config:_get_root_path:66 - Found root path: /Users/luke/Git/CryoGrid-run-manager/runs/abramov-tmp
2025-03-21 10:19:06.793 | SUCCESS  | cryogrid_pytools.excel_config:__init__:35 - Loaded CryoGrid Excel configuration file: /Users/luke/Git/CryoGrid-run-manager/runs/abramov-tmp/abramov-tmp.xlsx
2025-03-21 10:19:06.816 | INFO     | cryogrid_pytools.excel_config:check_strat_layers:466 - Checking stratigraphy layers...
2025-03-21 10:19:06.817 | SUCCESS  | cryogrid_pytools.excel_config:check_strat_layers:470 - [STRAT_layers_1]  parameters passed checks
2025-03-21 10:19:06.818 | SUCCESS  | cryogrid_pytools.excel_config:check_strat_layers:470 - [STRAT_layers_2]  parameters passed checks
2025-03-21 10:19:06.819 | SUCCESS  | cryogrid_pytools.excel_config:check_strat_layers:470 - [STRAT_layers_3]  parameters passed checks
2025-03-21 10:19:06.820 | INFO     | cryogrid_pytools.excel_config:__init__:50 - Start and end times: 1960-01-01 - 1970

(PosixPath('../runs/abramov-tmp/forcing/bbox.txt'),
 PosixPath('../runs/abramov-tmp/abramov-tmp.xlsx'))

In [45]:
def read_OUT_regridded_file(fname:str, deepest_point=None)->xr.Dataset:
    """
    Read a CryoGrid OUT_regridded_FCI2 file and return it as an xarray dataset.

    Parameters
    ----------
    fname : str
        Path to the .mat file
    deepest_point : float, optional
        Represents the deepest depth of the profile. If not provided, 
        then elevation is returned. Negative values represent depths below
        the surface.

    Returns
    -------
    ds : xarray.Dataset
        Dataset with dimensions 'time' and 'level'. The CryoGrid variable
        `depths` is renamed to `elevation`. If deepest_point is provided, then
        `depth` will represent the depth below the surface (negative below
        surface).
    """
    from cryogrid_pytools.matlab_helpers import read_mat_struct_flat_as_dict, matlab2datetime
    
    dat = read_mat_struct_flat_as_dict(fname)

    for key in dat:
        dat[key] = dat[key].squeeze()

    ds = xr.Dataset()
    ds.attrs['filename'] = fname

    times = matlab2datetime(dat.pop('timestamp'))
    depth = dat.pop('depths')

    for key in dat:
        ds[key] = xr.DataArray(
            data = dat[key].astype('float32'), 
            dims=['level', 'time'], 
            coords={'time': times})
        
    ds = ds.chunk(dict(time=-1))
    
    ds['depth'] = xr.DataArray(depth, dims=['level'])
    ds = ds.set_coords('depth')

    return ds


In [51]:
read_OUT_regridded_file('../runs/abramov-OUT_regridded/abramov-OUT_regridded_19610901.mat')

<xarray.Dataset> Size: 19kB
Dimensions:       (time: 2436, level: 0)
Coordinates:
  * time          (time) datetime64[ns] 19kB 1960-01-01T06:00:00 ... 1961-09-01
    depth         (level) uint8 0B 
Dimensions without coordinates: level
Data variables:
    T             (level, time) float32 0B dask.array<chunksize=(0, 2436), meta=np.ndarray>
    water         (level, time) float32 0B dask.array<chunksize=(0, 2436), meta=np.ndarray>
    ice           (level, time) float32 0B dask.array<chunksize=(0, 2436), meta=np.ndarray>
    class_number  (level, time) float32 0B dask.array<chunksize=(0, 2436), meta=np.ndarray>
Attributes:
    filename:  ../runs/abramov-OUT_regridded/abramov-OUT_regridded_19610901.mat